In [197]:
import wave
import requests
import time
import base64
from pyaudio import PyAudio, paInt16
import webbrowser
import serial
import modbus_tk
import modbus_tk.defines as cst
from modbus_tk import modbus_rtu
import os
from aip import AipSpeech
def ConnectRelay(PORT):
    """
    连接到继电器模块并执行基本的读取操作。
    
    :param PORT: 串行端口名称，例如 'COM3'
    :return: response_code 操作成功或失败的代码，master 485 主机对象或 None
    """
    try:
        # 创建串行连接对象，配置为 c2s03 设备默认的串行通信参数
        serial_connection = serial.Serial(
            port=PORT,
            baudrate=9600,
            bytesize=8,
            parity='E',
            stopbits=1
        )
        
        # 创建 modbus 连接对象
        master = modbus_rtu.RtuMaster(serial_connection)
        master.set_timeout(5.0)
        master.set_verbose(True)
        
        # 读取输入寄存器，c2s03 设备默认 slave=2, 起始地址=0, 输入寄存器个数 2
        master.execute(2, cst.READ_INPUT_REGISTERS, 0, 2)
        
        # 读取保持寄存器，c2s03 设备默认 slave=2, 起始地址=0, 保持寄存器个数 1
        # 可以根据需要修改保持寄存器的读取
        master.execute(2, cst.READ_HOLDING_REGISTERS, 0, 1)
        
        # 没有报错，返回 1
        response_code = 1
    
    except Exception as exc:
        # 打印异常信息
        print(str(exc))
        
        # 报错，返回 -1 并设置 master 为 None
        response_code = -1
        master = None
    
    return response_code, master

In [182]:
import sounddevice as sd
import wave
import numpy as np
import os

# 指定录音的参数
def record():
    SAMPLE_RATE = 16000  # 采样率，单位 Hz
    RECORD_SECONDS = 3   # 录音时长，单位 秒
    CHANNELS = 1         # 通道数，1 为单声道，2 为立体声
    BYTE_WIDTH = 2       # 位深，16位 PCM
    
    # 指定保存音频的文件夹路径
    folder_path = r"C:\Users\llq\Desktop\znxt"  # 替换为你的文件夹路径
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    file_path = os.path.join(folder_path, 'recorded_audio.wav')
    # 准备录音
    print('Recording started. It will automatically stop after {} seconds.'.format(RECORD_SECONDS))
    try:
        # 计算要录制的样本总数
        frames = int(SAMPLE_RATE * RECORD_SECONDS)
        # 使用 sounddevice 录制音频
        recording = sd.rec(frames, samplerate=SAMPLE_RATE, channels=CHANNELS, dtype='int16')
        # 启动录音并等待完成
        sd.wait()
    except KeyboardInterrupt:
        print(r'Recording stopped by user.')
    
    # 保存录音到文件
    with wave.open(file_path, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(BYTE_WIDTH)
        wf.setframerate(SAMPLE_RATE)
        wf.writeframes(recording.tobytes())
    
    print('Recording saved to:', file_path)

In [198]:
def change(result):
    ACTION = '000' 
    if result == '打开。':
        ACTION = 'ON'
    elif result == '关闭。':
        ACTION = 'OFF'
    return ACTION
def Switch(master, ACTION):
    """
    此函数为控制继电器开合函数，如果 ACTION=ON 则闭合，如果 ACTION=OFF 则断开。
    
    :param master: 485 主机对象，由 ConnectRelay 产生
    :param ACTION: ON 继电器闭合，开启风扇；OFF 继电器断开，关闭风扇。
    :return: >0 操作成功，<0 操作失败
    """
    try:
        # 将 ACTION 参数转换为小写，以实现不区分大小写的比较
        ACTION = ACTION.lower()

        # 如果 ACTION 包含 "on"，则闭合继电器
        if "on" in ACTION:
            # 写单个线圈，状态常量为 0xFF00，请求线圈接通
            # 设备默认 slave=2, 线圈地址=0, 请求线圈接通即 output_value 不等于 0
            master.execute(2, cst.WRITE_SINGLE_COIL, 0, output_value=1)
        else:
            # 如果 ACTION 包含 "off"，则断开继电器
            if "off" in ACTION:
                # 写单个线圈，状态常量为 0x0000，请求线圈断开
                # 设备默认 slave=2, 线圈地址=0, 请求线圈断开即 output_value 等于 0
                master.execute(2, cst.WRITE_SINGLE_COIL, 0, output_value=0)

        # 没有报错，返回 1
        response_code = 1

    except Exception as exc:
        # 打印异常信息
        print(str(exc))
        # 报错，返回 -1
        response_code = -1

    return response_code


In [222]:
a,master=ConnectRelay('COM3')
while True:
        record()
        # 百度智能云平台语音技能密钥
        BaiduAPP_ID = '84595183'
        BaiduAPI_KEY = 'JyaxymyHcw8PVCYiq1MMjH14'
        SECRET_KEY = '7sdcIb5B3Bdiwyq8VqZH4xEDGbjUsviX'
        client = AipSpeech(BaiduAPP_ID, BaiduAPI_KEY, SECRET_KEY)
        audio_file_path = r'C:\Users\llq\Desktop\znxt\recorded_audio.wav'
        # 检查文件是否存在
        if not os.path.isfile(audio_file_path):
            print("文件不存在，请检查文件路径和文件名。")
            print("文件路径：", audio_file_path)
        else:
            # 文件存在，读取本地音频文件
            try:
                with open(audio_file_path, 'rb') as f:
                    audio_data = f.read()
            except IOError as e:
                print(f"读取文件时发生错误: {e}")
            else:
                # 调用百度智能云语音识别接口
                result = client.asr(audio_data, 'wav', 16000, {
                    'dev_pid': 1537,  # 普通话识别
                })
                # 检查识别结果
                if 'result' in result:
                    print("识别结果：", result["result"][0])
                else:
                    print("识别失败，没有结果返回。")
        if not os.path.isfile(audio_file_path) or 'audio_data' not in locals():
            print("无法读取音频数据，语音识别未执行。")
        answer=change(result['result'][0])
        Switch(master,answer)
        a=input()
        if a=='0':
            break
        else:
            continue

Recording started. It will automatically stop after 3 seconds.
Recording saved to: C:\Users\llq\Desktop\znxt\recorded_audio.wav
识别结果： 大开。


 1213345657


Recording started. It will automatically stop after 3 seconds.
Recording saved to: C:\Users\llq\Desktop\znxt\recorded_audio.wav
识别结果： 打开。


 12134


Recording started. It will automatically stop after 3 seconds.
Recording saved to: C:\Users\llq\Desktop\znxt\recorded_audio.wav
识别结果： 这比较简单关闭。


 345678


Recording started. It will automatically stop after 3 seconds.
Recording saved to: C:\Users\llq\Desktop\znxt\recorded_audio.wav
识别结果： 关闭。


 0


1